## Hiperparâmetros com Bayesian Optimization

O ajuste de hiperpârametros em um modelo de **Machine Learning** é extremamente importante, já que pode elevar substancialmente o desempenho do modelo em aderência aos dados.
É muito comum ver em tutoriais de Machine Learning o uso do **Grid Search** para ajustar os hiperparâmetros, entranto há um problema de custo de processamento alto já que o Grid Search irá executar vários modelos e testar todo o intervalo de hiperparâmetros definido pelo usuário.
Em contra partida o **Bayesian Optimization** é um método de ajuste de hiperparâmetros que utiliza a tecnologia de **Bayesian Inference** para encontrar o melhor valor de hiperparâmetros para o modelo.


#### Estudo
Usaremos um estudo já abordado em um tutorial aqui no blog: [Trading Machine - Decision Tree](https://jhonattanln.github.io/2022/05/12/Decision_Tree.html)
Onde basicamente o objetivo é classificar se o retorno do ativo será positivou ou negativo ao longo dos dias.

### Bibliotecas utilizadas:

In [7]:
import pandas as pd
import numpy as np
import ta  ### biblioteca para analise de indicadores técnicos
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

### Base de dados:
A base de dados utilizada foi coletada da plataforma Economatica e contém:
- Fechamento do ativo: BOVA11
- Quantidade de negócios;
- Volume;
- Fechamento;
- Abertura;
- Mínima;
- Máxima;
- Médio.

In [3]:
df = pd.read_excel('economatica.xlsx', skiprows=3, index_col=0, parse_dates=True)
df.rename(columns={'Volume$': 'Volume'}, inplace=True)

### Features:
A biblioteca **ta** foi utilizada para extrair as features da base de dados.

In [4]:
df['Retornos'] = df.Fechamento.pct_change() ## retornos
df['Kama'] = ta.momentum.KAMAIndicator(close=df.Fechamento, window=21).kama() ## indicador Kama
df['ROC'] = ta.momentum.ROCIndicator(close=df.Fechamento, window=12).roc()
df['RSI'] = ta.momentum.RSIIndicator(close=df.Fechamento, window=14).rsi()
df['Stoch'] = ta.momentum.StochasticOscillator(close=df.Fechamento, high=df.Máximo, low=df.Mínimo, 
                                                window=14, smooth_window=3).stoch()
df['Chaikin_money'] = ta.volume.ChaikinMoneyFlowIndicator(high=df.Máximo, low=df.Mínimo, close=df.Fechamento, 
                                                          volume=df.Volume, window=20).chaikin_money_flow()
df['Force_index'] = ta.volume.ForceIndexIndicator(close=df.Fechamento, 
                                                  volume=df.Volume, window=13).force_index() 
df['Normal'] = (df.Fechamento - df.Mínimo) / (df.Máximo - df.Mínimo) 

### Tratando dados:
Houve a necessidade da limpeza de dados faltantes quando calculados os indicadores, como também a criação de targets para o modelo de classificação supervisionada

In [5]:
df = df.dropna() ## excluindo valores nulos
X = df[[
        'Q Negs', 'Q Títs', 'Volume', 'Fechamento', 'Abertura', 'Mínimo', 
        'Máximo', 'Médio', 'Kama', 'ROC', 'RSI', 'Stoch', 'Chaikin_money', 
        'Force_index', 'Normal']] ## criando as features
y = np.where(df['Fechamento'].shift(-1) > df['Fechamento'], 1, -1) ## criando target

Através train_test_split foi criado a base de treinamento e teste

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


O pipeline sera divididos em dois processos: i) Normalização dos dados para evitar distorções da amostra; ii) O algoritimo de Random Forest
>> Random forest é um algoritmo de Machine Learning muito utilizado, como um algoritimo de bagging tem essencialmente a ideia de diminuir a variância dos erros justamente fazendo uma interação na estrutura dos dados utilizando diferentes abordagens para isso.

In [9]:
scaler = StandardScaler()
dt = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('Scaler',scaler),
                    ('Decision_Tree', dt)])
                    
criterion = ["gini", 'entropy']
max_depth = [2, 4, 6, 8, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]
random_state = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
min_samples_split = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
min_samples_leaf = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
max_features = ['auto', 'sqrt', 'log2']

In [11]:
### Criando parametros para comparação 
parameters = dict(Decision_Tree__criterion=criterion, 
                    Decision_Tree__max_depth=max_depth,
                    Decision_Tree__random_state=random_state,
                    Decision_Tree__min_samples_split=min_samples_split,
                    Decision_Tree__min_samples_leaf=min_samples_leaf,
                    Decision_Tree__max_features=max_features)


In [14]:
def objective(params):
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return -MSE(y_test, y_pred)

In [15]:
bayes_opt = gp_minimize(objective, parameters, n_calls=100, random_state=0)

ValueError: Dimension has to be a list or tuple.